<a href="https://colab.research.google.com/github/Rocking-Priya/2025-summer-mod-6/blob/main/Mod_6_Homework_Reflection_Week_9_12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Homework Reflection 9

1. Write some code that will use a simulation to estimate the standard deviation of the coefficient when there is heteroskedasticity.  
Compare these standard errors to those found via statsmodels OLS or a similar linear regression model.

2. Write some code that will use a simulation to estimate the standard deviation of the coefficient when errors are highly correlated / non-independent.
Compare these standard errors to those found via statsmodels OlS or a similar linear regression model.

Show that if the correlation between coefficients is high enough, then the estimated standard deviation of the coefficient, using bootstrap errors,
might not match that found by a full simulation of the Data Generating Process.  (This can be fixed if you have a huge amount of data for the bootstrap simulation.)

Estimate standard error under heteroskedasticity

Heteroskedasticity = error variance changes with X or W.

In [1]:
import numpy as np
import statsmodels.api as sm

In [ ]:
import numpy as np
import statsmodels.api as sm

# Set simulation parameters
np.random.seed(42)
n_simulations = 1000
n = 1000
beta_estimates = []

for _ in range(n_simulations):
    W = np.random.normal(0, 1, n)
    X = W + np.random.normal(0, 1, n)
    # Heteroskedastic noise: depends on W
    noise = np.random.normal(0, 1 + 0.5 * np.abs(W))
    Y = 1 * X - W + noise

    # Add constant for OLS
    XW = sm.add_constant(np.column_stack([X, W]))
    model = sm.OLS(Y, XW).fit()
    beta_estimates.append(model.params[1])  # coefficient for X

# Simulation-based standard deviation of X's coefficient
simulated_std = np.std(beta_estimates)
print(f"Simulated std of beta (X): {simulated_std:.4f}")


Simulated std of beta (X): 0.0474


In [ ]:
print(np.std(beta_estimates))


0.047403283183051995


In [ ]:
from scipy.stats import skew
print(skew(beta_estimates))


-0.09132191293582789


Compare to statsmodels’ OLS standard error (one run)

In [ ]:
# Just one regression to compare OLS standard error
W = np.random.normal(0, 1, n)
X = W + np.random.normal(0, 1, n)
noise = np.random.normal(0, 1 + 0.5 * np.abs(W))
Y = 1 * X - W + noise

XW = sm.add_constant(np.column_stack([X, W]))
model = sm.OLS(Y, XW).fit()
ols_std = model.bse[1]  # standard error of X
print(f"OLS std of beta (X): {ols_std:.4f}")


OLS std of beta (X): 0.0439


Estimate std error when errors are correlated

Non-independent errors = correlation between nearby error terms.




In [ ]:
from scipy.stats import norm

beta_corr_estimates = []
for _ in range(n_simulations):
    W = np.random.normal(0, 1, n)
    X = W + np.random.normal(0, 1, n)

    # Correlated errors: add AR(1)-style autocorrelation
    epsilon = np.random.normal(0, 1, n)
    for i in range(1, n):
        epsilon[i] += 0.9 * epsilon[i-1]  # correlation with previous error

    Y = 1 * X - W + epsilon

    XW = sm.add_constant(np.column_stack([X, W]))
    model = sm.OLS(Y, XW).fit()
    beta_corr_estimates.append(model.params[1])

# Standard deviation from simulation
simulated_corr_std = np.std(beta_corr_estimates)
print(f"Simulated std with correlated errors: {simulated_corr_std:.4f}")


Simulated std with correlated errors: 0.0743


Show bootstrap error can mismatch under high collinearity

Let’s show if X and W are highly correlated, bootstrap errors might not match the truth.

In [ ]:
from sklearn.utils import resample

# Generate one dataset with highly correlated X and W
W = np.random.normal(0, 1, n)
X = W + np.random.normal(0, 0.01, n)  # very little noise → X ≈ W
Y = 1 * X - W + np.random.normal(0, 1, n)

# Run bootstrap
bootstrap_betas = []
for _ in range(500):
    Y_sample, X_sample, W_sample = resample(Y, X, W)
    XW_sample = sm.add_constant(np.column_stack([X_sample, W_sample]))
    model = sm.OLS(Y_sample, XW_sample).fit()
    bootstrap_betas.append(model.params[1])

# Bootstrap std
bootstrap_std = np.std(bootstrap_betas)
print(f"Bootstrap std with high collinearity: {bootstrap_std:.4f}")

# Compare to statsmodels
XW = sm.add_constant(np.column_stack([X, W]))
model = sm.OLS(Y, XW).fit()
ols_std = model.bse[1]
print(f"OLS std with high collinearity: {ols_std:.4f}")


Bootstrap std with high collinearity: 3.2284
OLS std with high collinearity: 3.2500


In [ ]:
n_simulations = 1000
significant_count = 0
n = 1000

for _ in range(n_simulations):
    # Simulate data using the same logic as in previous cells
    W = np.random.normal(0, 1, n)
    X = W + np.random.normal(0, 1, n)
    # Heteroskedastic noise: depends on W
    noise = np.random.normal(0, 1 + 0.5 * np.abs(W))
    Y = 1 * X - W + noise

    XW = sm.add_constant(np.column_stack([X, W]))
    model = sm.OLS(Y, XW).fit()

    t_value = model.tvalues[1]  # t-value of X
    if abs(t_value) > 1.96:
        significant_count += 1

power = significant_count / n_simulations
print(f"Power ≈ {power:.2%}")

Power ≈ 100.00%


In [ ]:


np.random.seed(42)

def simulate_power(B, A=1, C=10, D=1000, n_simulations=500):
    detected = 0
    for _ in range(n_simulations):
        W = np.random.normal(0, 1, D)
        X = W + np.random.normal(0, B, D)
        Y = A * X - W + np.random.normal(0, C, D)
        XW = sm.add_constant(np.column_stack([X, W]))
        model = sm.OLS(Y, XW).fit()
        t_value = model.tvalues[1]  # t-value for X
        if abs(t_value) > 1.96:
            detected += 1
    return detected / n_simulations  # return proportion detected

# Try different values of B
B_values = [0.2, 0.6, 1.8, 5.4]
for B in B_values:
    power = simulate_power(B)
    print(f"B = {B}: Detection rate ≈ {power:.2f}")


B = 0.2: Detection rate ≈ 0.11
B = 0.6: Detection rate ≈ 0.47
B = 1.8: Detection rate ≈ 1.00
B = 5.4: Detection rate ≈ 1.00


In [ ]:


np.random.seed(42)
n_simulations = 1000
n = 100  # D = 100

detection_rates = []

for A in [0.5, 1.0, 2.0, 4.0]:
    detections = 0
    for _ in range(n_simulations):
        W = np.random.normal(0, 1, n)
        X = W + np.random.normal(0, 1, n)  # B = 1
        Y = A * X - W + np.random.normal(0, 10, n)  # C = 10

        XW = sm.add_constant(np.column_stack([X, W]))
        model = sm.OLS(Y, XW).fit()
        t_value = model.tvalues[1]  # t-stat for X
        if abs(t_value) > 1.96:
            detections += 1
    detection_rate = detections / n_simulations
    detection_rates.append((A, detection_rate))
    print(f"A = {A}, Detection rate = {detection_rate:.2f}")


A = 0.5, Detection rate = 0.08
A = 1.0, Detection rate = 0.16
A = 2.0, Detection rate = 0.51
A = 4.0, Detection rate = 0.98



Homework Reflection 11

1. Construct a dataset for an event study where the value, derivative, and second derivative of a trend all change discontinuously (suddenly) after an event.
Build a model that tries to decide whether the event is real (has a nonzero effect) using:
(a) only the value,
(b) the value, derivative, and second derivative.
Which of these models is better at detecting and/or quantifying the impact of the event?  (What might "better" mean here?)

2. Construct a dataset in which there are three groups whose values each increase discontinuously (suddenly) by the same amount at a shared event; they change in parallel
over time, but they have different starting values.  Create a model that combines group fixed effects with an event study, as suggested in the online reading.
Explain what you did, how the model works, and how it accounts for both baseline differences and the common event effect.

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf

# Step 1: Simulate the dataset for event study with discontinuous changes
np.random.seed(0)
time = np.arange(20)
event_time = 10

# Pre-event trend (linear)
pre_event = time[:event_time]
post_event = time[event_time:]

# Create values with jump in value, slope (1st derivative), and acceleration (2nd derivative)
y = np.concatenate([
    2 * pre_event + 5,                                # before event: linear trend
    2 * post_event + 5 + 10 + 3 * (post_event - 10) + 2 * (post_event - 10)**2  # after event: jump + new slope + curve
])

df_event = pd.DataFrame({'time': time, 'y': y})

# Add 1st and 2nd derivative manually (finite difference approximations)
df_event['dy'] = df_event['y'].diff().fillna(0)
df_event['d2y'] = df_event['dy'].diff().fillna(0)

# Step 2: Create models
# a) Model with only y
df_event['event'] = (df_event['time'] >= event_time).astype(int)
model_y = smf.ols('y ~ event', data=df_event).fit()

# b) Model with y, dy, and d2y
model_full = smf.ols('event ~ y + dy + d2y', data=df_event).fit()

# Output summaries
model_y_summary = model_y.summary()
model_full_summary = model_full.summary()

df_event.head(), model_y_summary, model_full_summary


(   time   y   dy  d2y  event
 0     0   5  0.0  0.0      0
 1     1   7  2.0  2.0      0
 2     2   9  2.0  0.0      0
 3     3  11  2.0  0.0      0
 4     4  13  2.0  0.0      0,
 <class 'statsmodels.iolib.summary.Summary'>
 """
                             OLS Regression Results                            
 Dep. Variable:                      y   R-squared:                       0.523
 Model:                            OLS   Adj. R-squared:                  0.496
 Method:                 Least Squares   F-statistic:                     19.73
 Date:                Wed, 23 Jul 2025   Prob (F-statistic):           0.000315
 Time:                        01:09:56   Log-Likelihood:                -105.80
 No. Observations:                  20   AIC:                             215.6
 Df Residuals:                      18   BIC:                             217.6
 Df Model:                           1                                         
 Covariance Type:            nonrobust           

In [6]:
import pandas as pd
import statsmodels.formula.api as smf

In [4]:
df_10 = pd.read_csv('/content/homework_10.1.csv')
df_10.head()

,Unnamed: 0,city,time,X,y
0,0,0,0,0.144044,7.552716
1,1,0,1,1.454274,10.077829
2,2,0,2,0.761038,12.372731
3,3,0,3,0.121675,11.489263
4,4,0,4,0.443863,13.104833


In [7]:
# Create a model: y depends on city and time
model = smf.ols('y ~ C(city) + C(time)', data=df_10).fit()

# Show the results
print(model.summary())


                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.866
Model:                            OLS   Adj. R-squared:                  0.858
Method:                 Least Squares   F-statistic:                     109.2
Date:                Wed, 23 Jul 2025   Prob (F-statistic):          2.09e-134
Time:                        01:13:19   Log-Likelihood:                -700.90
No. Observations:                 360   AIC:                             1444.
Df Residuals:                     339   BIC:                             1525.
Df Model:                          20                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept         6.6763      0.422     15.820

In [8]:
# Extract time fixed effects
time_effects = model.params.filter(like='C(time)')

# Show all fixed effects for time
print(time_effects)


C(time)[T.1]     2.252281
C(time)[T.2]     3.804417
C(time)[T.3]     4.618865
C(time)[T.4]     5.492543
C(time)[T.5]     6.562369
C(time)[T.6]     6.190129
C(time)[T.7]     5.755543
C(time)[T.8]     5.515259
C(time)[T.9]     5.255354
C(time)[T.10]    2.960641
C(time)[T.11]    1.931654
dtype: float64


In [9]:
# Add the baseline effect (intercept) to all time fixed effects
intercept = model.params['Intercept']
all_times = {}

# Time 0 = intercept only
all_times[0] = intercept

# Time 1 to 11 = intercept + coefficient
for t in range(1, 12):
    coef_name = f'C(time)[T.{t}]'
    all_times[t] = intercept + model.params.get(coef_name, 0)

# Turn it into a readable DataFrame
df_time_effects = pd.DataFrame.from_dict(all_times, orient='index', columns=['Fixed Effect'])
df_time_effects.index.name = 'Time'
print(df_time_effects)


      Fixed Effect
Time              
0         6.676278
1         8.928559
2        10.480695
3        11.295143
4        12.168821
5        13.238647
6        12.866408
7        12.431822
8        12.191537
9        11.931632
10        9.636919
11        8.607932
